In [1]:
import pandas as pd

# Set display options to show more all columns
pd.set_option('display.max_columns', None) 

listings = pd.read_csv('../AirBnb datasets/barcelona/listings.csv')

/Users/krys/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


Split 'amenities' column into individual amenities

In [2]:
listings['amenities'] = listings['amenities'].str.split(', ')

# Create a new boolean feature for each amenity
amenities_dummies = listings['amenities'].str.join('|').str.get_dummies()
amenities_dummies.columns = amenities_dummies.columns.str.strip().str.replace('"', '', regex=False)
amenities_dummies.columns = [col.replace(' ', '_') for col in amenities_dummies.columns]
amenities_dummies.columns = [col.replace('[', '').replace(']', '') for col in amenities_dummies.columns]

amenities_dummies.head()

Wifi_Home_Connect  iQ700_stainless_steel_oven  shampoo  \
0                  0                           0        0   
1                  0                           0        0   
2                  0                           0        0   
3                  0                           0        0   
4                  0                           0        0   

   _Bont\u00e9_or_similar/Shampoo_+_Conditioner_shampoo  _Neutro__body_soap  \
0                                                  0                      0   
1                                                  0                      0   
2                                                  0                      0   
3                                                  0                      0   
4                                                  0                      0   

   -_body_soap  ._body_soap  ._conditioner  ._refrigerator  ._shampoo  \
0            0            0              0               0          0   
1            0            0              0               0          0   
2            0            0              0               0          0   
3            0            0              0               0          0   
4            0            0              0               0          0   

   .._oven  .._refrigerator  0%_body_soap  0%_shampoo  \
0        0                0             0           0   
1        0                0             0           0   
2        0                0             0           0   
3        0                0             0           0   
4        0                0             0           0   

   1_bote_grande_basico_al_entrar._No_se_repone_despues._shampoo  1_inch_HDTV  \
0                                                  0                        0   
1                                                  0                        0   
2                                                  0                        0   
3                                                  0                        0   
4                                                  0                        0   

   10_inch_HDTV  10_inch_TV_with_standard_cable  \
0             0                               0   
1             0                               0   
2             0                               0   
3             0                               0   
4             0                               0   

   100_inch_HDTV_with_premium_cable  120_inch_HDTV_with_standard_cable  \
0                                 0                                  0   
1                                 0                                  0   
2                                 0                                  0   
3                                 0                                  0   
4                                 0                                  0   

   125_inch_HDTV_with_Netflix  125_inch_HDTV  130_inch_HDTV  14_inch_TV  \
0                           0              0              0           0   
1                           0              0              0           0   
2                           0              0              0           0   
3                           0              0              0           0   
4                           0              0              0           0   

   15_inch_TV_with_Netflix  15_inch_TV_with_standard_cable  15_inch_TV  \
0                        0                               0           0   
1                        0                               0           0   
2                        0                               0           0   
3                        0                               0           0   
4                        0                               0           0   

   17_inch_HDTV  18_inch_TV_with_standard_cable  \
0             0                               0   
1             0                               0   
2             0                               0   
3             0                         

Aggregate columns

In [3]:
import re
from collections import defaultdict

def normalize_column_name(column_name):
    return re.sub(r'^[^a-zA-Z0-9]+|[^a-zA-Z0-9]+$', '', column_name).lower()

# Dictionary to hold groups of columns based on normalized names
column_groups = defaultdict(list)

# Normalize names
for col in amenities_dummies.columns:
    normalized_name = normalize_column_name(col)
    column_groups[normalized_name].append(col)

aggregated_columns = {}
columns_to_drop = set()
for normalized_name, columns in column_groups.items():
    if len(columns) > 1:
        aggregated_column_name = f"{normalized_name}"
        amenities_dummies[aggregated_column_name] = amenities_dummies[columns].sum(axis=1)
        aggregated_columns[aggregated_column_name] = columns
        columns_to_drop.update(columns)

amenities_dummies.drop(columns=columns_to_drop, inplace=True)

def replace_greater_than_one(x):
    return 1 if x > 1 else x

amenities_dummies = amenities_dummies.applymap(replace_greater_than_one)

amenities_dummies.head()

/var/folders/hr/dg_n9mds23z57wvq3v_cd7340000gn/T/ipykernel_23127/3443501558.py:20: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  amenities_dummies[aggregated_column_name] = amenities_dummies[columns].sum(axis=1)
/var/folders/hr/dg_n9mds23z57wvq3v_cd7340000gn/T/ipykernel_23127/3443501558.py:20: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  amenities_dummies[aggregated_column_name] = amenities_dummies[columns].sum(axis=1)
/var/folders/hr/dg_n9mds23z57wvq3v_cd7340000gn/T/ipykernel_23127/3443501558.py:20: PerformanceWarning: DataFra

Wifi_Home_Connect  iQ700_stainless_steel_oven  \
0                  0                           0   
1                  0                           0   
2                  0                           0   
3                  0                           0   
4                  0                           0   

   _Bont\u00e9_or_similar/Shampoo_+_Conditioner_shampoo  _Neutro__body_soap  \
0                                                  0                      0   
1                                                  0                      0   
2                                                  0                      0   
3                                                  0                      0   
4                                                  0                      0   

   0%_body_soap  0%_shampoo  \
0             0           0   
1             0           0   
2             0           0   
3             0           0   
4             0           0   

   1_bote_grande_basico_al_entrar._No_se_repone_despues._shampoo  1_inch_HDTV  \
0                                                  0                        0   
1                                                  0                        0   
2                                                  0                        0   
3                                                  0                        0   
4                                                  0                        0   

   10_inch_HDTV  10_inch_TV_with_standard_cable  \
0             0                               0   
1             0                               0   
2             0                               0   
3             0                               0   
4             0                               0   

   100_inch_HDTV_with_premium_cable  120_inch_HDTV_with_standard_cable  \
0                                 0                                  0   
1                                 0                                  0   
2                                 0                                  0   
3                                 0                                  0   
4                                 0                                  0   

   125_inch_HDTV_with_Netflix  125_inch_HDTV  130_inch_HDTV  14_inch_TV  \
0                           0              0              0           0   
1                           0              0              0           0   
2                           0              0              0           0   
3                           0              0              0           0   
4                           0              0              0           0   

   15_inch_TV_with_Netflix  15_inch_TV_with_standard_cable  15_inch_TV  \
0                        0                               0           0   
1                        0                               0           0   
2                        0                               0           0   
3                        0                               0           0   
4                        0                               0           0   

   17_inch_HDTV  19_inch_TV_with_premium_cable  2_Westinghouse_conditioner  \
0             0                              0                           0   
1             0                              0                           0   
2             0                              0                           0   
3             0                              0                           0   
4             0                              0                           0   

   2_neveras_1,86_cm_marca_ok_refrigerator  2_unidades_refrigerator  \
0                                        0                        0   
1                                        0                        0   
2                                        0                        0   
3                                        0                        0   
4                                        0                        0  

Selected top 25 amenities

In [4]:
amenities_final = amenities_dummies[['wifi', 'heating', 'essentials', 'kitchen', 'smoke_alarm', 
                                     'air_conditioning', 'hangers', 'coffee_maker', 'dedicated_workspace', 
                                     'hot_water_kettle', 'hair_dryer', 'iron', 'tv', 'hot_water', 
                                     'washer', 'dryer', 'first_aid_kit', 'fire_extinguisher', 
                                     'refrigerator', 'pool', 'Gym', 'pets_allowed', 
                                     'free_parking_on_premises', 'paid_parking_on_premises', 'self_check-in']]

amenities_final.head()

,wifi,heating,essentials,kitchen,smoke_alarm,air_conditioning,hangers,coffee_maker,dedicated_workspace,hot_water_kettle,hair_dryer,iron,tv,hot_water,washer,dryer,first_aid_kit,fire_extinguisher,refrigerator,pool,Gym,pets_allowed,free_parking_on_premises,paid_parking_on_premises,self_check-in
0,1,1,1,1,0,0,1,1,1,0,1,1,1,1,1,0,0,0,1,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0
2,1,1,1,1,0,0,1,1,0,0,1,1,0,1,0,0,0,0,1,0,0,0,0,1,0
3,1,1,1,1,1,1,1,0,0,1,1,1,0,1,1,0,0,1,1,0,0,1,0,0,0
4,1,0,1,1,1,0,1,0,1,1,1,1,0,1,0,0,0,1,1,0,0,0,0,0,0


In [5]:
amenities_final.to_csv('amenities.csv', index=False)